# ECE-6524 / CS-6524 Deep Learning
# Assignment 3 [100 pts]

In this assignment, **you need to complete the Yolo loss function, and train an object detector. Yay!**

## Submission guideline for the coding part (Jupyter Notebook)

1. Click the Save button at the top of the Jupyter Notebook
2. Please make sure to have entered your Virginia Tech PID below
3. Once you've completed everything (make sure output for all cells are visible), select File -> Download as -> PDF via LaTeX
4. Look at the PDF file and make sure all your solutions are displayed correctly there 
7. Zip all the files along with this notebook (Please don't include the data). Name it as Assignment_3_Code_[YOUR PID NUMBER].zip
8. Name your PDF file as Assignment_2_NB_[YOUR PID NUMBER].pdf
9. **<span style="color:blue"> Submit your zipped file and the PDF SEPARATELY**</span>

Note: if facing issues with step 3 refer: https://pypi.org/project/notebook-as-pdf/

## Submission guideline for the coding part (Google Colab)

1. Click the Save button at the top of the Notebook
2. Please make sure to have entered your Virginia Tech PID below
3. Follow last two cells in this notebook for guidelines to download pdf file of this notebook
4. Look at the PDF file and make sure all your solutions are displayed correctly there 
5. Zip all the files along with this notebook (Please don't include the data). Name it as Assignment_2_Code_[YOUR PID NUMBER].zip
6. Name your PDF file as Assignment_2_NB_[YOUR PID NUMBER].pdf
7. **<span style="color:blue"> Submit your zipped file and the PDF SEPARATELY**</span>

**While you are encouraged to discuss with your peers, <span style="color:blue">all work submitted is expected to be your own.</span> <span style="color:red">If you use any information from other resources (e.g. online materials), you are required to cite it below you VT PID. Any violation will result in a 0 mark for the assignment.</span>**

### Please Write Your VT PID Here: 906255893
### Reference (if any): 

https://araintelligence.com/blogs/deep-learning/object-detection/yolo_v1/

https://www.kdnuggets.com/2018/05/implement-yolo-v3-object-detector-pytorch-part-1.html

https://github.com/pjreddie/darknet/issues/558

https://stats.stackexchange.com/questions/333968/understanding-the-yolo-algorithm

https://zhuanlan.zhihu.com/p/139713442

https://github.com/weiaicunzai/Pytorch-yolo/blob/master/YOLOLoss.py

https://github.com/z-huabao/pytorch-yolov1/blob/master/yoloLoss.py

https://github.com/Eversee22/yolov1-pytorch/blob/master/loss.py

https://github.com/motokimura/yolo_v1_pytorch/blob/master/loss.py

https://github.com/xiongzihua/pytorch-YOLO-v1/blob/master/yoloLoss.py

In this homework, you would need to use **Python 3.6+** along with the following packages:
```
1. pytorch 1.2
2. torchvision
3. numpy
4. matplotlib
5. tqdm (for better, cuter progress bar. Yay!)
```
To install pytorch, please follow the instructions on the [Official website](https://pytorch.org/). In addition, the [official document](https://pytorch.org/docs/stable/) could be very helpful when you want to find certain functionalities. 


<span style="color:red">Note that, on a high-end GPU, it sill takes 3-4 hours to train. **SO START EARLY. IT'S IMPOSSIBLE TO FINISH IT AT THE LAST MINUTE!**</span>


### Colab Setup

### To select GPU in Google Colab:
- go to **Edit -> Notebook settings -> Hardware accelerator -> GPU**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
# modify "customized_path_to_homework", path of folder in drive, where you uploaded your homework
customized_path_to_homework = "/content/drive/My Drive/DL_Fall_2020/Assignment_3"
sys.path.append(customized_path_to_homework)

In [ ]:
# run this to download dataset, give path to the download.sh file from your drive
!sh /content/drive/My\ Drive/DL_Fall_2020/Assignment_3/download_data.sh

--2020-10-16 17:54:51--  http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar [following]
--2020-10-16 17:54:52--  https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/octet-stream]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

rainval_06-Nov-2007   0%[                    ]   3.21M   234KB/s    eta 32m 37s^C


In [ ]:
# copy and place downloaded dataset to your drive. To access dataset multiple times, no need to download everytime you open colab.
!cp -r /content/VOCdevkit_2007 enter_path_to_your_folder_here (example: /content/drive/My\ Drive/DL_Fall_2020/Assignment_3/)

In [ ]:
import os
import random
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import models

from resnet_yolo import resnet50
from dataset import VocDetectorDataset
from eval_voc import evaluate
from predict import predict_image
from config import VOC_CLASSES, COLORS
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Initialization

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name()) # GPU name

Tesla P4


# You Only Look Once: Unified, Real-Time Object Detection 
In this assignment, you need to implement the loss function and train the **YOLO object detector** (specfically, YOLO-v1). Here we provide a list of recommend readings for you:
- [YOLO original paper](https://arxiv.org/pdf/1506.02640.pdf) (recommended)
- [Object detection methods](http://slazebni.cs.illinois.edu/fall18/lec09_detection.pdf) (Slides)
- [Great post about YOLO](https://medium.com/adventures-with-deep-learning/yolo-v1-part-1-cfb47135f81f) on Medium
- [Differences between YOLO, YOLOv2 and YOLOv3
](https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088)
- [Great explanation of the Yolo Loss function](https://stats.stackexchange.com/questions/287486/yolo-loss-function-explanation)

We adopt a variant of YOLO, which:
1. Use pretrained ResNet50 classifier as detector backbone. The pretrained model is offered in `torchvision.models`.
2. Instead of using a $7\times7$ detection grid, we use $14\times14$ to get a more finegrained detection.

In general, the backbone models are usually pretrained on ImageNet dataset (> 1 million images) with numerous classes. As a result, having these pretrained backbone can greatly shorten the required training time, as well as improve the performance. <span style="color:red">**But still, it takes at least 3-4 hours to train, not to mention that you might need to debug after one training run. So START EARLY, DON'T GO #YOLO!**</span>

<img src="figure/example.png" width="450">
You are supposed to get a reasonable detector (like the ... above?) after training the model correctly.

In [ ]:
# YOLO network hyperparameters
B = 2  # number of bounding box predictions per cell
S = 14  # width/height of network output grid (larger than 7x7 from paper since we use a different network)

## Load the pretrained ResNet classifier
Load the pretrained classifier. By default, it would use the pretrained model provided by `Pytorch`.

In [ ]:
load_network_path = None
pretrained = True

# use to load a previously trained network
if load_network_path is not None:
    print('Loading saved network from {}'.format(load_network_path))
    net = resnet50().to(device)
    net.load_state_dict(torch.load(load_network_path))
else:
    print('Load pre-trained model')
    net = resnet50(pretrained=pretrained).to(device)

Load pre-trained model


Some basic hyperparameter settings that you probably don't have to tune.

In [ ]:
learning_rate = 0.001
num_epochs = 20
batch_size = 10

# Yolo loss component coefficients (as given in Yolo v1 paper)
lambda_coord = 5
lambda_noobj = 0.5

## Implement the YOLO-v1 loss [80 pts]
Now, you have to implement the `YoloLoss` for training your object detector. Please read closely to the [YOLO original paper](https://arxiv.org/pdf/1506.02640.pdf) so that you can implement it.

In general, there are 4 components in the YOLO loss. Consider that we have our prediction grid of size$(N, S, S, 5B+c)$ ( (x, y, w, h, C) for each bounding box, and c is the number of classes), where $N$ is the batch size, $S$ is the grid size, $B$ is the number of bounding boxes. We have :
1. Bounding box regression loss on the bounding box$(x, y, w, h)$
    - $l_{coord}=\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{1}^{obj}_{ij}\left[(x_i-\hat{x}_i)^2+(y_i-\hat{y}_i)^2\right]$ + $\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{1}^{obj}_{ij}\left[(\sqrt{w_i}-\sqrt{\hat{w}_i})^2+(\sqrt{h_i}-\sqrt{\hat{h}_i})^2\right]$
    - $\mathbb{1}^{obj}_{ij}$: equals to 1 when object appears in cell $i$, and the bounding box $j$ is responsible for the prediction. 0 otherwise.
2. Contain object loss on the confidence prediction $c$ (only calculate for those boxes that actually have objects)
    - $l_{contain}=\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{1}^{obj}_{ij}(C_i-\hat{C}_i)^2$
    - $C_i$ the predicted confidence score for cell $i$ from predicted box $j$
    - For each grid cell, you only calculate the contain object loss for the predicted bounding box that has maximum overlap (iou) with the gruond truth box.
    - We say that this predicted box with maximum iou is **responsible** for the prediction.
3. No object loss on the confidence prediction $c$ (only calculate for those boxes that don't have objects)
    - $l_{noobj}=\sum_{i=0}^{S^2}\sum_{j=0}^B\mathbb{1}^{noobj}_{ij}(C_i-\hat{C}_i)^2$
    - $\mathbb{1}^{obj}_{ij}$: equals to 1 when **no object appears** in cell $i$.
4. Classification error loss.
    - $l_{class}=\sum_{i=0}^{S^2}\mathbb{1}_i^{obj}\sum_{c\in classes}\left(p_i(c)-\hat{p_i}(c)\right)^2$
    - $p_i(c)$ is the predicted score for class $c$
    
Putting them together, we get the yolo loss:
\begin{equation}
yolo=\lambda_{coord}l_{coord}+l_{contain}+\lambda_{noobj}l_{noobj}+l_{class}
\end{equation}
where $\lambda$ are hyperparameters. We have provided detailed comments to guide you through implementing the loss. So now, please complete the YoloLoss in the code block below. **If you have any problem with regard to implementation, post and discuss it on Piazza.**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
 
class YoloLoss(nn.Module):
    def __init__(self,S,B,lambda_coord,lambda_noobj): #this was incorrectly assigned so I fixed it for future lines
        super(YoloLoss,self).__init__()
        self.S = S    # square grid size
        self.B = B    # number of bounding boxes per grid cell 
        self.lambda_coord = lambda_coord     # localization penalty dampener/amplifier in loss function
        self.lambda_noobj = lambda_noobj     # detection penalty dampener/amplifier in loss function
 
    def compute_iou(self, box1, box2):                                                                                                                                                             
        """Compute the intersection over union of two set of boxes, each box is [x1,y1,x2,y2].
        Args:
          box1: (tensor) bounding boxes, sized [N,4].
          box2: (tensor) bounding boxes, sized [M,4].
        Return:
          (tensor) iou, sized [N,M].
        """
        N = box1.size(0)  # first size element of bounding box 1
        M = box2.size(0)  # first size element of bounding box 2

        # Finds top left coordinate of each bounding box
        lt = torch.max(
            box1[:,:2].unsqueeze(1).expand(N,M,2),  # [N,2] -> [N,1,2] -> [N,M,2]
            box2[:,:2].unsqueeze(0).expand(N,M,2),  # [M,2] -> [1,M,2] -> [N,M,2]
        )   
 
        # Finds bottom right coordinate of each bounding box
        rb = torch.min(
            box1[:,2:].unsqueeze(1).expand(N,M,2),  # [N,2] -> [N,1,2] -> [N,M,2]
            box2[:,2:].unsqueeze(0).expand(N,M,2),  # [M,2] -> [1,M,2] -> [N,M,2]
        )   
 
        # finds the width and height of IoU
        wh = rb - lt  # [N,M,2] 
        wh[wh<0] = 0  # clip at 0 (crop out negative values which imply image boundaries)
        inter = wh[:,:,0] * wh[:,:,1]  # [N,M] #Area of IoU
 
        area1 = (box1[:,2]-box1[:,0]) * (box1[:,3]-box1[:,1])  # [N,] # Area of box 1 (x2-x1)*(y2-y1)
        area2 = (box2[:,2]-box2[:,0]) * (box2[:,3]-box2[:,1])  # [M,] # Area of box 2 (x2-x1)*(y2-y1)
        area1 = area1.unsqueeze(1).expand_as(inter)  # [N,] -> [N,1] -> [N,M]
        area2 = area2.unsqueeze(0).expand_as(inter)  # [M,] -> [1,M] -> [N,M]
 
        iou = torch.true_divide(inter,(area1 + area2 - inter)) # IoU ratio, replaced division as it was deprecated
        return iou 
    
    def get_class_prediction_loss(self, classes_pred, classes_target):
        """ 
        Parameters:
        classes_pred : (tensor) size (batch_size, S, S, 20)                                                                                                                                        
        classes_target : (tensor) size (batch_size, S, S, 20)
         
        Returns:
        class_loss : scalar
        """
        ##### CODE #####
        class_loss = nn.MSELoss(classes_pred,classes_target, reduction = 'sum') # Classification error loss, assuming classes_pred and classes_target are already in probability format 
        ##### CODE #####
        return class_loss
         
         
    def get_regression_loss(self, box_pred_response, box_target_response):
        """
        Parameters:
        box_pred_response : (tensor) size (-1, 5)
        box_target_response : (tensor) size (-1, 5)
        Note : -1 corresponds to ravels the tensor into the dimension specified 
        See : https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view_as
         
        Returns:
        reg_loss : scalar
        """
        ##### CODE #####
        # regresslion loss for localization (centerpoint offset penalty + height and width penalty)
        reg_loss = nn.MSELoss(box_pred_response[:,:2],box_target_response[:,:2],reduction='sum') + nn.MSELoss(torch.sqrt(box_pred_response[:,2:4]),torch.sqrt(box_target_response[:,2:4]),reduction='sum')
        ##### CODE #####
        return reg_loss
         
    def get_contain_object_loss(self, box_pred_response, box_target_response_iou):
        """
        Parameters:
        box_pred_response : (tensor) size ( -1 , 5)
        box_target_response_iou : (tensor) size ( -1 , 5)
        Note : -1 corresponds to ravels the tensor into the dimension specified 
        See : https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view_as
         
        Returns:
        contain_loss : scalar
        """
        ##### CODE #####
        # object detection penalty
        contain_loss = MSELoss(box_pred_response[:,4],box_target_response_iou[:,4],reduction='sum') 
        ##### CODE #####
        return contain_loss
         
    def get_no_object_loss(self, target_tensor, pred_tensor, no_object_mask):
        """                                                                                                                                                                                        
        Parameters:
        target_tensor : (tensor) size (batch_size, S , S, 30)
        pred_tensor : (tensor) size (batch_size, S , S, 30)
        no_object_mask : (tensor) size (batch_size, S , S)
         
        Returns:
        no_object_loss : scalar
         
        Hints:
        1) Create 2 tensors no_object_prediction and no_object_target which only have the 
        values which have no object. 
        2) Have another tensor no_object_prediction_mask of the same size such that 
        mask with respect to both confidences of bounding boxes set to 1. 
        3) Create 2 tensors which are extracted from no_object_prediction and no_object_target using
        the mask created above to find the loss. 
        """
        ##### CODE #####
        # Tensor for values that have no object in prediction
        no_object_prediction = pred_tensor[no_object_mask].view(-1,30)
        # Tensor for values that have no object in actual data
        no_object_target = target_tensor[no_object_mask].view(-1,30)
        # Creating tensor to hold prediction mask which tells each layer which of the bounding boxes it is responsible for predicting
        no_object_prediction_mask = torch.cuda.ByteTensor(no_object_prediction.size())
        # Zeroing out created tensor
        no_object_prediction_mask.zero_()
        # Setting mask on relevant bounding boxes to 1
        no_object_prediction_mask[:,4]=1
        no_object_prediction_mask[:,9]=1
        # Applying mask to prediction and actual
        no_object_prediction_1 = no_object_prediction[no_object_prediction_mask]
        no_object_target_1 = no_object_target[no_object_prediction_mask]
        # Creating no object loss per grid cell
        no_object_loss = nn.MSELoss(no_object_prediction_1,no_object_target_1,reduction='sum')
        ##### CODE #####
        return no_object_loss
          
    def find_best_iou_boxes(self, box_target, box_pred):
        """
        Parameters: 
        box_target : (tensor)  size (-1, 5)
        box_pred : (tensor) size (-1, 5)
        Note : -1 corresponds to ravels the tensor into the dimension specified 
        See : https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view_as
         
        Returns: 
        box_target_iou: (tensor)
        contains_object_response_mask : (tensor)
         
        Hints:
        1) Find the iou's of each of the 2 bounding boxes of each grid cell of each image.
        2) Set the corresponding contains_object_response_mask of the bounding box with the max iou
        of the 2 bounding boxes of each grid cell to 1.
        3) For finding iou's use the compute_iou function (done)
        4) Before using compute preprocess the bounding box coordinates in such a way that 
        if for a Box b the coordinates are represented by [x, y, w, h] then 
        x, y = x/S - 0.5*w, y/S - 0.5*h ; w, h = x/S + 0.5*w, y/S + 0.5*h
        Note: Over here initially x, y are the center of the box and w,h are width and height. 
        We perform this transformation to convert the correct coordinates into bounding box coordinates. (done)
        5) Set the confidence of the box_target_iou of the bounding box to the maximum iou (done)
        """
        ##### CODE #####
        # Creating tensor to hold prediction mask which tells each layer which of the bounding boxes it is responsible for predicting
        contains_object_response_mask = torch.cuda.ByteTensor(box_target.size())
        # Zeroing out created tensor
        contains_object_response_mask.zero_()
        # Creating tensor to hold prediction mask which tells each layer which of the bounding boxes it is responsible for predicting
        not_contains_object_response_mask = torch.cuda.ByteTensor(box_target.size())
        # Zeroing out created tensor
        not_contains_object_response_mask.zero_()
        # Creating tensor to hold IoU
        box_target_iou = torch.zeros(box_target.size()).cuda()
        # for loop over each two proposed bounding boxes in each grid cell
        for i in range(0,box_pred.size(0),self.B): 
            box_1 = box_pred[i:i+2] # proposed boxes in examined grid cell
            # create tensor to hold grid-relative coordinates (YOLO bounding box encoding)
            box_1_coords = Variable(torch.FloatTensor(box_1.size()))
            # Adjust centerpoint to be relative to grid
            box_1_coords[:,:2] = box_1[:,:2]/self.S -0.5*box_1[:,2:4]  # x, y
            box_1_coords[:,2:4] = box_1[:,:2]/self.S +0.5*box_1[:,2:4] # w, h
            # Reshape actual box to vector
            box_2 = box_target[i].view(-1,5)
            # create tensor to hold grid-relative coordinates (YOLO bounding box encoding)
            box_2_coords = Variable(torch.FloatTensor(box2.size()))
            # Adjust centerpoint to be relative to grid
            box_2_coords[:,:2] = box_2[:,:2]/self.S -0.5*box_2[:,2:4] # x, y
            box_2_coords[:,2:4] = box_2[:,:2]/self.S +0.5*box_2[:,2:4] # w, h
            iou = self.compute_iou(box_1_coords[:,:4],box_2_coords[:,:4]) # using previous fn to compute IoU ratio
            max_iou, max_index = iou.max(0)
            max_index = max_index.data.cuda() # convert to cuda for GPU computation
            # pick the box with the larger IoU ratio
            contains_object_response_mask[i+max_index]=1
            # pick box with smaller IoU ratio
            not_contains_object_response_mask[i+1-max_index]=1
            # Set IoUs for each true bounding box in the image
            box_target_iou[i+max_index,torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
        box_target_iou = Variable(box_target_iou).cuda()
        coo_response_mask = contains_object_response_mask
        ##### CODE #####
        return box_target_iou, coo_response_mask
         
    def forward(self, pred_tensor,target_tensor):
        '''
        pred_tensor: (tensor) size(batchsize,S,S,Bx5+20=30)
                      where B - number of bounding boxes this grid cell is a part of = 2
                            5 - number of bounding box values corresponding to [x, y, w, h, c]
                                where x - x_coord, y - y_coord, w - width, h - height, c - confidence of having an object
                            20 - number of classes
         
        target_tensor: (tensor) size(batchsize,S,S,30)
         
        Returns:
        Total Loss
        '''
        N = pred_tensor.size(0) # size of input
         
        total_loss = None #initialization of variable
        # Create 2 tensors contains_object_mask and no_object_mask 
        # of size (Batch_size, S, S) such that each value corresponds to if the confidence of having 
        # an object > 0 in the target tensor.

        ##### CODE #####
        contains_object_mask = target_tensor[:,:,:,4] > 0
        no_object_mask = target_tensor[:,:,:,4] == 0
        contains_object_mask = contains_object_mask.unsqueeze(-1).expand_as(target_tensor)
        no_object_mask = no_object_mask.unsqueeze(-1).expand_as(target_tensor)
        ##### CODE #####                      
        
        """
        Create a tensor contains_object_pred that corresponds to 
        to all the predictions which seem to confidence > 0 for having an object
        Then, split this tensor into 2 tensors :                                                                                                                                                       
        1) bounding_box_pred : Contains all the Bounding box predictions (x, y, w, h, c) of all grid 
                                cells of all images
        2) classes_pred : Contains all the class predictions for each grid cell of each image
        Hint : Use contains_object_mask
        """ 
        ##### CODE #####
        contains_object_pred = pred_tensor[contains_object_mask].view(-1,30)
        bounding_box_pred = contains_object_pred[:,:10].contiguous().view(-1,5) 
        classes_pred = contains_object_pred[:,10:] 
        ##### CODE #####                   
        """
        # Similarly, create 2 tensors bounding_box_target and classes_target
        # using the contains_object_mask.
        """
        ##### CODE #####
        contains_object_target = target_tensor[contains_object_mask].view(-1,30)
        bounding_box_target = contains_object_target[:,:10].contiguous().view(-1,5)
        classes_target = contains_object_target[:,10:]
        ##### CODE #####
        
        #Compute the No object loss here
        # Instruction: finish your get_no_object_loss
        ##### CODE #####
        no_object_loss = self.get_no_object_loss(target_tensor, pred_tensor, no_object_mask)
        ##### CODE #####
        """
        # Compute the iou's of all bounding boxes and the mask for which bounding box 
        # of 2 has the maximum iou the bounding boxes for each grid cell of each image.
        # Instruction: finish your find_best_iou_boxes and use it.
        """
        ##### CODE #####
        best_iou_boxes = self.find_best_iou_boxes(bounding_box_target, bounding_box_pred)[0]
        ##### CODE #####
        """        
        # Create 3 tensors :
        # 1) box_prediction_response - bounding box predictions for each grid cell which has the maximum iou
        # 2) box_target_response_iou - bounding box target ious for each grid cell which has the maximum iou
        # 3) box_target_response -  bounding box targets for each grid cell which has the maximum iou
        # Hint : Use coo_response_mask
        """
        ##### CODE #####
        box_prediction_response = bounding_box_pred[self.find_best_iou_boxes(bounding_box_target, bounding_box_pred)[1]].view(-1,5)
        box_target_response_iou = best_iou_boxes[self.find_best_iou_boxes(bounding_box_target, bounding_box_pred)[1]].view(-1,5)
        box_target_response = bounding_box_target[self.find_best_iou_boxes(bounding_box_target, bounding_box_pred)[1]].view(-1,5) 
        ##### CODE #####
        """
        # Find the class_loss, containing object loss and regression loss
        """
        ##### CODE #####
        # computing final pieces of loss function
        localization_loss = self.lambda_coord*(self.get_regression_loss(box_prediction_response, box_target_response))
        detection_loss = self.get_contain_object_loss(box_prediction_response, box_target_response_iou)+self.lambda_noobj*self.get_no_object_loss(target_tensor, pred_tensor, no_object_mask)
        class_prediction_loss = self.get_class_prediction_loss(classes_pred, classes_target)
        total_loss = localization_loss + detection_loss + class_prediction_loss
        ##### CODE #####
        return total_loss / N 

In [ ]:
criterion = YoloLoss(S, B, lambda_coord, lambda_noobj)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

## Reading Pascal Data

Since Pascal is a small dataset (5000 in train+val) we have combined the train and val splits to train our detector. This is not typically a good practice, but we will make an exception in this case to be able to get reasonable detection results with a comparatively small object detection dataset. Use `download_data.sh` to download the dataset.

The train dataset loader also using a variety of data augmentation techniques including random shift, scaling, crop, and flips. Data augmentation is slightly more complicated for detection dataset since the bounding box annotations must be kept consistent through the transformations.

Since the output of the dector network we train is a $(S, S, 5B+c)$ tensor, we use an encoder to convert the original bounding box coordinates into relative grid bounding box coordinates corresponding to the the expected output. We also use a decoder which allows us to convert the opposite direction into image coordinate bounding boxes.

In [ ]:
file_root_train = customized_path_to_homework + '/VOCdevkit_2007/VOC2007/JPEGImages/'
annotation_file_train = customized_path_to_homework + '/voc2007.txt'

train_dataset = VocDetectorDataset(root_img_dir=file_root_train,dataset_file=annotation_file_train,train=True, S=S)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
print('Loaded %d train images' % len(train_dataset))

Initializing dataset
Loaded 5011 train images


In [ ]:
file_root_test = customized_path_to_homework + '/VOCdevkit_2007/VOC2007test/JPEGImages/'
annotation_file_test = customized_path_to_homework + '/voc2007test.txt'

test_dataset = VocDetectorDataset(root_img_dir=file_root_test,dataset_file=annotation_file_test,train=False, S=S)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
print('Loaded %d test images' % len(test_dataset))

Initializing dataset
Loaded 4950 test images


## Train detector
Now, train your detector.

In [ ]:
best_test_loss = np.inf

for epoch in range(num_epochs):
    net.train()
    
    # Update learning rate late in training
    if epoch == 30 or epoch == 40:
        learning_rate /= 10.0

    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    
    print('\n\nStarting epoch %d / %d' % (epoch + 1, num_epochs))
    print('Learning Rate for this epoch: {}'.format(learning_rate))
    
    total_loss = 0.
    
    for i, (images, target) in enumerate(tqdm(train_loader, total=len(train_loader))):
        images, target = images.to(device), target.to(device)

        pred = net(images)
        loss = criterion(pred,target)
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [%d/%d], average_loss: %.4f'
            % (epoch+1, num_epochs, total_loss / (i+1)))
    
    # evaluate the network on the test data
    with torch.no_grad():
        test_loss = 0.0
        net.eval()
        for i, (images, target) in enumerate(tqdm(test_loader, total=len(test_loader))):
            images, target = images.to(device), target.to(device)

            pred = net(images)
            loss = criterion(pred,target)
            test_loss += loss.item()
        test_loss /= len(test_loader)
    
    if best_test_loss > test_loss:
        best_test_loss = test_loss
        print('Updating best test loss: %.5f' % best_test_loss)
        torch.save(net.state_dict(),'best_detector.pth')

    torch.save(net.state_dict(),'detector.pth')

  0%|          | 0/502 [00:00<?, ?it/s]



Starting epoch 1 / 20
Learning Rate for this epoch: 0.001


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:20.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:130: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:20.)
  0%|          | 0/502 [00:04<?, ?it/s]


RuntimeError: ignored

# View example predictions

Now, take a glance at how your detector works:

In [ ]:
net.eval()
net.load_state_dict(torch.load('best_detector.pth'))
# select random image from train set
image_name = random.choice(train_dataset.fnames)
image = cv2.imread(os.path.join(file_root_train, image_name))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
threshold = 0.1
print('predicting...')
print(image.shape)
result = predict_image(net, image_name, root_img_directory=file_root_train, threshold=threshold)
for left_up, right_bottom, class_name, _, prob in result:
    color = COLORS[VOC_CLASSES.index(class_name)]
    cv2.rectangle(image, left_up, right_bottom, color, 2)
    label = class_name + str(round(prob, 2))
    text_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
    p1 = (left_up[0], left_up[1] - text_size[1])
    cv2.rectangle(image, (p1[0] - 2 // 2, p1[1] - 2 - baseline), (p1[0] + text_size[0], p1[1] + text_size[1]),
                  color, -1)
    cv2.putText(image, label, (p1[0], p1[1] + baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, 8)

plt.figure(figsize = (15,15))
plt.imshow(image)

## Evaluate on Test [20 pts]

To evaluate detection results we use mAP (mean of average precision over each class), You are expected to get an map of at least 49.

In [ ]:
test_aps = evaluate(net, test_dataset_file=annotation_file_test, threshold=threshold)

### Guidelines for Downloading PDF in Google Colab
- Run below cells only in Google Colab, Comment out in case of Jupyter notebook

In [ ]:
#Run below two lines (in google colab), installation steps to get .pdf of the notebook

!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

# After installation, comment above two lines and run again to remove installation comments from the notebook.

In [ ]:
# Find path to your notebook file in drive and enter in below line

!jupyter nbconvert --to PDF "your_notebook_path_here/DL_Assignment_2.ipynb"

#Example: "/content/drive/My Drive/DL_Fall_2020/Assignment_2/DL_Assignment_2.ipynb"